# CHiME‑9 Task 1 – Subtask 5: Videoanalyse‑Pipeline

Dieses Notebook dokumentiert und demonstriert die Video‑ und Blickanalyse‑Pipeline für Subtask 5 der CHiME‑9 Task 1 (MCoRec).  
Der Fokus liegt auf der Ableitung von Sprecher‑Clusterings aus Video‑basierten Signalen (Seating, ASD‑Tracks, Kopf‑Yaw/Gaze) und auf der Auswertung der resultierenden Cluster im Kontext der Challenge‑Daten. Das Notebook basiert auf diesem Branch: https://github.com/AudioVisual-Projekt/auvis/tree/Feature/task_5_alternative_cluster

Die Notebook‑Demo zeigt:

- wie die zentrale Inferenz (`main.py`) für Gesprächs‑Clustering aufgerufen wird,
- wie nachgelagerte Gaze‑ und Distanzberechnungen mit `post_step.py` funktionieren,
- wie Video‑Cluster mit `eval_video_cluster_quality.py` bewertet werden,
- wie ASD‑Overlap‑ und Seating‑Baselines evaluiert werden,
- wie die resultierenden JSON‑ und NPY‑Outputs weiter analysiert und visualisiert wurden(z. B. Distanz‑Histogramme und Correct‑Rate‑Kurven).


In [ ]:
# Installations- und Umgebungs-Hinweise

from pathlib import Path
import sys
#
# Projektstruktur (relevante Ausschnitte): 
# auvis/
#   team_c/
#     main.py
#     post_step.py
#     plots_py.py
#     eval_asd_overlap_baseline.py
#     eval_seating.py
#     eval_video_cluster_quality.py
#     eval_sessions.py
#     data-bin/
#       dev/
#         sessionXXX/
#       output/
#       output_gaze/
#       output_gaze_plots_spectral/
#         speaker_clustering_eval.json
#         speaker_distance_matrices.json
#         gaze_distance_gt_bins/...

# === Pfadkonfiguration für das Notebook ===

# Passe diesen Pfad an dein lokales Repo an (Ordner, der 'team_c' enthält).
PROJECT_ROOT = Path("/path/to/auvis/team_c").resolve()

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Daten-Wurzel (analog zu den Skripten, die mit 'data-bin' arbeiten).
DATABIN_ROOT = PROJECT_ROOT / "data-bin"

DEV_SESSIONS_ROOT = DATABIN_ROOT / "dev"
OUTPUT_ROOT = DATABIN_ROOT / "output"
OUTPUT_GAZE_ROOT = DATABIN_ROOT / "output_gaze"
OUTPUT_GAZE_PLOTS_SPECTRAL = DATABIN_ROOT / "output_gaze_plots_spectral"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATABIN_ROOT:", DATABIN_ROOT)


## Projektüberblick & Main-Skript (`main.py`)

Das Skript `main.py` ist der zentrale Entry‑Point für das zeit‑ und semantikbasierte Gesprächs‑Clustering in Subtask 5.[file:8]  
Es definiert eine Klasse `InferenceEngine`, die für jede MCoRec‑Session Metadaten lädt, Sprechersegmente aus ASD‑Tracks ableitet und mehrere Clustering‑Varianten erzeugt.

Die wichtigsten Verarbeitungsschritte innerhalb von `InferenceEngine.mcorec_session_infer(...)` sind:
1. Laden der Session‑Metadaten (`metadata.json`) und Sammeln aller ASD‑Tracks pro Sprecher.
2. Ableitung von Sprecher‑Aktivitätssegmenten via `get_speaker_activity_segments` aus `src.cluster.convspks` (Zeitsegmente im UEM‑Fenster).
3. Berechnung von Gesprächs‑Scores pro Sprecherpaar mit `calculate_conversation_scores` und zeitbasiertes Agglomerativ‑Clustering (`clusterspeakers`), das nach `speakertocluster_time.json` geschrieben wird.
4. Rein semantisches Clustering mit `semantic_cluster_speakers`, das semantische Embeddings aus `team_c/data-bin` nutzt und `speakertocluster_semantic.json` erzeugt. 
5. Hybrid‑Clustering, das Zeit‑ und Semantik‑Informationen kombiniert (`hybrid_cluster_speakers`) und `speakertocluster_hybrid.json` schreibt.

Die `main(...)`‑Funktion parst ein Session‑Glob (`--sessiondir`), iteriert über alle passenden Session‑Ordner, legt pro Session ein Output‑Verzeichnis unter `team_c/data-bin/<outputdirname>/<session>` an und ruft für jede Session `InferenceEngine.mcorec_session_infer(...)` auf.


In [ ]:
# Minimaler Wrapper um die Main-Pipeline (Zeit/Semantik/Hybrid) 
from glob import glob
import os

from main import InferenceEngine  # Modul liegt im PROJECT_ROOT (team_c). 


def run_time_semantic_hybrid_clustering(
    session_glob: str,
    output_dirname: str = "output_semantic",
    max_length: int = 15,
) -> None:
    """
    Run the unified time / semantic / hybrid clustering pipeline for all
    sessions matching a given glob pattern (relative to PROJECT_ROOT).

    This is a thin wrapper around main.py that mirrors its behavior without
    changing any core logic.
    """
    session_pattern = os.path.join(str(PROJECT_ROOT), session_glob)
    session_dirs = [p for p in glob(session_pattern) if os.path.isdir(p)]
    session_dirs = sorted(session_dirs)

    if not session_dirs:
        print("No sessions found for pattern:", session_pattern)
        return

    engine = InferenceEngine(max_length=max_length)  # same default as in main.py. 

    output_base = DATABIN_ROOT / output_dirname
    output_base.mkdir(parents=True, exist_ok=True)

    print(f"Running inference for {len(session_dirs)} sessions...")
    for session_dir in session_dirs:
        session_name = os.path.basename(os.path.normpath(session_dir))
        output_dir = output_base / session_name
        output_dir.mkdir(parents=True, exist_ok=True)

        print(f"\nSession: {session_name}")
        print("  session_dir:", session_dir)
        print("  output_dir :", output_dir)

        # Delegate all heavy lifting to the original InferenceEngine. 
        engine.mcorec_session_infer(session_dir=session_dir, output_dir=str(output_dir))



## Utility-Module: I/O, Pfade und Basis-Helfer (`post_step.py`)

Das Skript `post_step.py` implementiert einen eigenständigen Post‑Processing‑Schritt, der auf bereits erzeugten Session‑Outputs aufsetzt (Clusterings, Sitzgeometrie, Gaze‑Tracks).
Es enthält zunächst generische Helfer für JSON‑I/O (`read_json`, `write_json`), Pfad‑Discovery (`discover_teamc_root`) und Normalisierung von Sprecher‑IDs (`norm_spk_id`, `intersection_keys`).

Diese Utilities werden in den späteren Funktionen zur Distanzberechnung, zu agglomerativem Clustering und zu spektraler Gaze‑Analyse wiederverwendet.
Für das Notebook ist es daher sinnvoll, die zentralen Helfer in einer kompakten Form zu importieren und bei Bedarf zu nutzen.


In [ ]:
from typing import Dict, List, Optional, Tuple

import numpy as np
from pathlib import Path

import post_step as pst   


def read_json(path: Path):
    """Thin wrapper around post_step.read_json for convenient use in the notebook."""
    return pst.read_json(path)  # type: ignore[attr-defined]


def write_json(path: Path, obj) -> None:
    """Thin wrapper around post_step.write_json for convenient use in the notebook."""
    return pst.write_json(path, obj)  # type: ignore[attr-defined]


def discover_teamc_root(start: Path) -> Path:
    """
    Discover the 'team_c' repo root by walking up from 'start',
    mirroring post_step.discover_teamc_root. [file:3]
    """
    return pst.discover_teamc_root(start)  # type: ignore[attr-defined]


def norm_spk_id(x) -> str:
    """
    Normalize speaker identifiers to 'spk<int>' IDs, reusing the original helper. [file:3]
    """
    return pst.norm_spk_id(x)  # type: ignore[attr-defined]


def intersection_keys(a: Dict[str, int], b: Dict[str, int]) -> List[str]:
    """Return sorted intersection of dict keys (speaker IDs). [file:3]"""
    return pst.intersection_keys(a, b)  # type: ignore[attr-defined]


## Utility-Module: ASD-, Gaze- und Geometrie-Verarbeitung (`post_step.py`)

Im nächsten Block von `post_step.py` finden sich Loader und Helfer für Sitzgeometrie, ASD‑Zuordnungen und Gaze‑Statistiken:

- `load_seat_geometry(npz_path)`: lädt `seatgeometry.npz` mit normalisierter Sitzdistanzmatrix `dist_seat`, Sitzwinkeln `theta_deg` und `person_ids`. 
- `load_asd_mapping(asd_json_path)`: lädt `asdseatmatching.json` und erzeugt eine robuste Abbildung von ASD‑Track‑IDs auf Personen‑IDs, inkl. Fallback‑Logik über `assignments_all`.[file:3]  
- `load_gaze(gaze_path)`: liest `gazetracks.json` und liefert für jede Person Median‑Yaw, IQR, Stichprobenanzahl und optional die volle Yaw‑Sample‑Serie.
- Zirkuläre Hilfsfunktionen `circ_abs_diff_deg` und `circ_signed_diff_deg` behandeln Winkel auf dem Kreis korrekt.

Diese Funktionen kapseln die Rohdaten‑Formate der CHiME‑9‑Video‑Annotationsdateien und sind die Basis für die spätere Distanz‑ und Similaritätsberechnung zwischen Sprechern.[file:3]


In [ ]:
# Nutzung der Geometrie- und Gaze-Utilities in einem Beispiel-Wrapper
def load_session_geometry_and_gaze(session_dir: Path):
    """
    Load seat geometry, ASD mapping and gaze tracks for a single session.

    Expected files in 'session_dir':
    - seatgeometry.npz
    - asdseatmatching.json
    - gazetracks.json (optional) [file:3]
    """
    seat_npz = session_dir / "seatgeometry.npz"
    asd_json = session_dir / "asdseatmatching.json"
    gaze_json = session_dir / "gazetracks.json"

    if not seat_npz.exists() or not asd_json.exists():
        raise FileNotFoundError("Missing seatgeometry.npz or asdseatmatching.json in "
                                f"{session_dir}")

    geom = pst.load_seat_geometry(seat_npz)  # type: ignore[attr-defined]
    asd_ids, asd_to_person = pst.load_asd_mapping(asd_json)  # type: ignore[attr-defined]

    if gaze_json.exists():
        gaze_by_person = pst.load_gaze(gaze_json)  # type: ignore[attr-defined]
    else:
        gaze_by_person = {}

    return geom, asd_ids, asd_to_person, gaze_by_person


## Utility-Module: Sprecher-Distanzmatrizen & Gaze-Integration (`post_step.py`)

Auf Basis von Sitzgeometrie, ASD‑Mapping und Gaze‑Daten baut `build_speaker_distance_matrices(...)` drei M×M‑Matrizen (M = Anzahl ASD‑Tracks):

- `D_seat`: reine Sitzdistanz (0..1) entlang der kreisförmigen Sitzordnung.
- `D_seat_gaze`: Sitzdistanz, skaliert mit einem kontinuierlichen Gaze‑Interaktionsfaktor, der mutual/one‑way Gaze mit Gewichten `WMUTUAL` und `WONEWAY` einbezieht.
- `D_gaze_only`: reine Interaktionsdistanz (Basis 1.0), reduziert durch Gaze‑Kontakt; wird 0 bei starker gegenseitiger Blickinteraktion.

Die Funktion erzeugt zusätzlich ein `meta`‑Dictionary mit Sprecherreihenfolge, Sitzzuordnung, Gaze‑Nutzungsstatistik und zusammengefassten Metriken wie quantilen von mutual/avg‑look‑Werten.
Diese Daten werden später sowohl für agglomeratives Clustering als auch für spektrale Gaze‑Graphen verwendet und global in `speaker_distance_matrices.json` aggregiert.


In [ ]:
# Beispiel: Distanzmatrizen für eine Session berechnen und inspizieren 

from pprint import pprint


def build_session_distance_matrices(session_dir: Path):
    """
    Compute D_seat, D_seat_gaze and D_gaze_only for a single session
    and return them together with the meta information. [file:3]
    """
    geom, asd_ids, asd_to_person, gaze_by_person = load_session_geometry_and_gaze(
        session_dir
    )

    d_seat, d_seat_gaze, d_gaze_only, meta = pst.build_speaker_distance_matrices(
        geom=geom,
        asdids=asd_ids,
        asdtoperson=asd_to_person,
        gazebyperson=gaze_by_person,
    )  # type: ignore[attr-defined]

    print("Speaker order:", meta.get("speakerorder"))
    print("D_seat shape      :", d_seat.shape)
    print("D_seat_gaze shape :", d_seat_gaze.shape)
    print("D_gaze_only shape :", d_gaze_only.shape)

    return d_seat, d_seat_gaze, d_gaze_only, meta



## Utility-Module: Agglomeratives Clustering & Spektrale Gaze-Graphen (`post_step.py`)

Für die Distanzmatrizen implementiert `post_step.py` mehrere Clustering‑Bausteine:

- `agglo_labels_from_precomputed_D(D, k)`: Agglomeratives Clustering mit average‑Linkage auf einer vorkomputierten Distanzmatrix (`metric="precomputed"` bzw. älteren `affinity`‑API‑Fallback).[file:3]  
- `choose_k_without_gt(D, kmin, kmax)`: Auswahl von k allein aus D durch Maximierung des Silhouette‑Scores; k wird auf \([2, \min(k_{\max}, n)]\) beschränkt. 

Für Gaze‑basierte Graphen werden Similaritätsmatrizen `S` berechnet:

- `build_mutual_similarity_knn(meta, knn, gamma)`: mutual‑Gaze‑Graph, Similarität \(\text{mutual} = \min(p_{i\to j}, p_{j\to i})\), mit Potenzschärfung `gamma` und optionaler kNN‑Sparsifizierung.  
- `build_similarity_avg_knn(meta, knn, gamma)`: Similarität \(\text{avglook} = 0.5(p_{i\to j} + p_{j\to i})\) als weniger strikte Alternative.

Die spektralen Varianten wählen k über eine gewichtete Modularity‑Metrik auf dem Similaritätsgraphen:

- `spectral_cluster_auto_k(S, kmin, kmax)` liefert Clusterlabels und ein Info‑Dict mit `chosen_k`, `scores` und Parametern. 
- `evaluate_spectral_mutual(...)` und `evaluate_spectral_similarity(...)` wenden diese Strategie auf alle Sessions an, schreiben pro Session `speakertocluster_spectral*.json` und sammeln ARI/F1‑Scores sowie Graph‑Statistiken.

In [ ]:
# Beispiel: k-Auswahl und spektrales Clustering für eine Session

def spectral_clustering_for_session(session_dir: Path):
    """
    Run mutual- and avg-based spectral clustering for a single session,
    mirroring the logic inside post_step.evaluate_spectral_mutual /
    evaluate_spectral_similarity, but keeping side-effects minimal. [file:3]
    """
    meta_path = session_dir / "speaker_distance_meta.json"
    if not meta_path.exists():
        raise FileNotFoundError("speaker_distance_meta.json not found in "
                                f"{session_dir}. Did you run post_step.build_distance_matrices?")

    meta = read_json(meta_path)
    spk_order = meta.get("speakerorder", [])
    if not spk_order or len(spk_order) < 2:
        print("Not enough speakers for spectral clustering.")
        return

    n = len(spk_order)
    print("Number of speakers:", n)

    # Mutual-gaze similarity graph and clustering. 
    s_mutual = pst.build_mutual_similarity_knn(  # type: ignore[attr-defined]
        meta=meta,
        knn=pst.SPECTRAL_KNN,       # uses the same constants as in post_step.py
        gamma=pst.SPECTRAL_GAMMA,
    )
    labels_mutual, info_mutual = pst.spectral_cluster_auto_k(  # type: ignore[attr-defined]
        S=s_mutual,
        kmin=pst.SPECTRAL_KMIN,
        kmax=pst.SPECTRAL_KMAX,
    )
    print("Mutual-gaze chosen k:", info_mutual.get("chosenk"))

    # Avg-look similarity graph and clustering. [file:3]
    s_avg = pst.build_similarity_avg_knn(  # type: ignore[attr-defined]
        meta=meta,
        knn=pst.SPECTRAL_KNN,
        gamma=pst.SPECTRAL_GAMMA,
    )
    labels_avg, info_avg = pst.spectral_cluster_auto_k(  # type: ignore[attr-defined]
        S=s_avg,
        kmin=pst.SPECTRAL_KMIN,
        kmax=pst.SPECTRAL_KMAX,
    )
    print("Avg-look chosen k:", info_avg.get("chosenk"))

    return {
        "speakerorder": spk_order,
        "labels_mutual": labels_mutual,
        "labels_avg": labels_avg,
        "info_mutual": info_mutual,
        "info_avg": info_avg,
    }



## Post-Processing Main-Step (`post_step.main`) und globale Eval-JSONs

Die `main()`‑Funktion von `post_step.py` ist als eigenständiger Run‑Step konzipiert, der typischerweise nach der Inferenz mit `main.py` ausgeführt wird.
Sie findet automatisch das `team_c`‑Root, setzt `data-bin` als Daten‑ und Output‑Wurzel (`output_gaze` für Gaze‑Outputs, Labels‑Wurzel für GT) und führt dann vier Hauptschritte durch:

1. `evaluate_existing_assignments`: Bewertung vorhandener Sitz‑Baselines (`speakertocluster_seat*.json`) gegen Ground‑Truth‑Labels, Ergebnis u. a. in `speaker_clustering_eval.json` mit per‑Session‑ und Durchschnittsmetriken. 
2. `build_distance_matrices`: Erzeugt pro Session `speaker_distance_seat.npy`, `speaker_distance_seatgaze.npy`, `speaker_distance_gazeonly.npy` plus `speaker_distance_meta.json` sowie einen globalen Dump `speaker_distance_matrices.json`.
3. `evaluate_agglo_sweep`: Agglomeratives Clustering für `seat`, `seatgaze` und `gazeonly` mit automatischer k‑Auswahl via Silhouette; schreibt `speakertocluster_agglo*.json` und fasst ARI/F1‑Scores zusammen.
4. `evaluate_spectral_mutual` und `evaluate_spectral_similarity`: spektrale Gaze‑Cluster, k‑Auswahl über gewichtete Modularity; schreibt per‑Session‑Predictions und fasst Metriken in `speaker_clustering_eval.json`.


In [ ]:
# Kompletten Post-Processing-Step ausführen 

def run_post_step(
    output_root: Optional[Path] = None,
    labels_root: Optional[Path] = None,
) -> dict:
    """
    Run the full post-processing pipeline on existing per-session outputs.

    This is a convenience wrapper that delegates to post_step.main(), which
    internally:
      - evaluates existing seat-based baselines,
      - builds distance matrices (seat, seat+gaze, gaze-only),
      - runs agglomerative clustering sweeps,
      - runs spectral clustering on gaze graphs,
      - writes summary JSONs 'speaker_clustering_eval.json' and
        'speaker_distance_matrices.json'. [file:3][file:9][file:10]
    """
    # The original main() uses global constants OUTPUT_ROOT and LABELS_ROOT
    # if set; here we mimic that behavior by temporarily overriding them. 
    if output_root is not None:
        pst.OUTPUT_ROOT = output_root  # type: ignore[attr-defined]
    if labels_root is not None:
        pst.LABELS_ROOT = labels_root  # type: ignore[attr-defined]

    pst.main()  # type: ignore[attr-defined]

    # Optionale Rückgabe: globaler Distance-Dump, falls vorhanden. 
    global_dump_path = OUTPUT_GAZE_ROOT / "speaker_distance_matrices.json"
    if global_dump_path.exists():
        return read_json(global_dump_path)

    return {}


## Video-Cluster-Qualität, ASD-Overlap-Baseline & Seating-Evaluation

Zusätzlich zu `main.py` und `post_step.py` gibt es drei Evaluationsskripte, die speziell für die Analyse von Video‑basierten Clustern relevant sind.

1. **ASD-Overlap-Baseline (`eval_asd_overlap_baseline.py`)** 
   - Sucht pro Session ASD‑JSONs unter `speakers/spk*/centralcrops/*asd.json`, leitet UEM‑Fenster aus Frame‑IDs ab und baut Sprachsegmente mit `get_speaker_activity_segments`.
   - Führt dann ein Agglomerativ‑Clustering mit k = Anzahl GT‑Cluster durch und bewertet F1/ARI gegen `labels/speakertocluster.json`.
   - Schreibt Predictions nach `data-bin/dev/output_asdoverlap/<session>/speakertocluster_asdoverlap.json` und fasst Metriken über Sessions hinweg zusammen.

2. **Seating-basierte Baseline-Evaluation (`eval_seating.py`)**
   - Liest pro Session `labels/speakertocluster.json` und verschiedene sitzbasierte Heuristik‑Outputs (`speakertocluster_seat_neighbors.json`, `..._opposites.json`, `..._halves.json`, `..._distcomponents.json`, `..._distk2.json`).
   - Berechnet F1 und ARI via `pairwise_f1_score` und `adjusted_rand_score` und gibt per‑Session sowie globale Mittelwerte aus.

3. **Video-Cluster-Quality (`eval_video_cluster_quality.py`)**
   - Lädt für jede Session `A.npy` (Seat‑Similaritätsmatrix) und `asdseatmatching.json`, konstruiert daraus mehrere Sitz‑Cluster‑Varianten (Nachbarn, Hälften, Opposites, hierarchisches k=2).
   - Mappt Sitz‑Cluster via ASD‑Matching auf Speaker‑Cluster, baut aus `A` eine Similaritätsmatrix im Sprecherraum und bewertet diese Cluster sowohl über interne Metriken (Silhouette, within‑vs‑between‑Quality) als auch gegen GT (F1/ARI), falls vorhanden.
   - Schreibt eine zusammenfassende JSON‑Datei `output/videovs_asd_eval.json` und optional eine TSV‑Übersicht pro Session.



In [ ]:
# Wrapper für die Evaluationsskripte (ASD-Overlap, Seating, Video-Qualität)

import eval_asd_overlap_baseline as eval_asd
import eval_seating as eval_seat
import eval_video_cluster_quality as eval_vid


def run_asd_overlap_baseline():
    """
    Run the ASD-overlap baseline evaluation over all dev sessions.

    Internally this:
      - discovers all 'data-bin/dev/session*' folders,
      - runs overlap-based clustering per session,
      - writes predictions under 'data-bin/dev/output_asdoverlap',
      - prints per-session and global F1/ARI scores. [file:4]
    """
    eval_asd.main()  # type: ignore[attr-defined]


def run_seating_baseline_eval():
    """
    Evaluate seating-based heuristics (neighbors, opposites, halves,
    distance components, distk2) against GT speaker clusters. [file:5]
    """
    eval_seat.main()  # type: ignore[attr-defined]


def run_video_cluster_quality_eval():
    """
    Evaluate video-based seat similarity A.npy vs ASD overlap and GT clusters.

    The script:
      - iterates over 'data-bin/dev/session*',
      - runs ASD-overlap baseline per session,
      - evaluates several seat-based clusterings in the speaker space,
      - writes JSON and TSV summaries to 'data-bin/output'. [file:6]
    """
    eval_vid.main()  # type: ignore[attr-defined]


## Visualisierung der Gaze- und Cluster-Statistiken (`plots_py.py` und Outputs)

Das Skript `plots_py.py` liest aggregierte JSON‑Files wie `speaker_clustering_eval.json`, `speaker_distance_matrices.json` und ggf. zusätzliche Zusammenfassungen, um verschiedene Analyseplots zu erzeugen.
Typische Beispiele sind Histogramme der Gaze‑basierten Distanzen (z. B. „ALL SESSIONS | distance_seat_gaze“) und Correct‑Rate‑Kurven in Abhängigkeit von einem Distanz‑Schwellwert („Correct‑Rate vs Threshold | distance_spectral_similarity“), wie in den beigefügten Abbildungen zu sehen.

Die dafür genutzten JSON‑Strukturen enthalten:

- pro Session: Sprecherreihenfolge, Distanzmatrizen und per‑Approach‑Clustering‑Metriken,
- globale Auswertungen über alle Sessions, etwa mittlere ARI/F1‑Werte je Ansatz (Baseline‑Distanzen, Agglo‑Varianten, spektrale Gaze‑Cluster) und Binned‑Statistiken für Gaze‑Distanzen versus GT‑Clusterzugehörigkeit.


In [ ]:
# Beispiel: Laden der globalen Speaker-Distance- und Clustering-Evals für Plotting 

import json


def load_global_distance_and_eval():
    """
    Load the global distance and clustering evaluation JSONs that are
    typically used for plotting. [file:3][file:9][file:10][file:14]
    """
    dist_path = OUTPUT_GAZE_PLOTS_SPECTRAL / "speaker_distance_matrices.json"
    eval_path = OUTPUT_GAZE_PLOTS_SPECTRAL / "speaker_clustering_eval.json"
    summary_path = OUTPUT_GAZE_PLOTS_SPECTRAL / "summary.json"

    data = {}

    if dist_path.exists():
        with dist_path.open("r", encoding="utf-8") as f:
            data["distance_matrices"] = json.load(f)

    if eval_path.exists():
        with eval_path.open("r", encoding="utf-8") as f:
            data["clustering_eval"] = json.load(f)

    if summary_path.exists():
        with summary_path.open("r", encoding="utf-8") as f:
            data["summary"] = json.load(f)

    print("Loaded keys:", list(data.keys()))
    return data

 


## End-to-End-Demonstration: Mini-Pipeline-Run für eine Session

In diesem Abschnitt wird eine typische Ausführungsreihenfolge für Subtask 5 demonstriert:

1. Zeit-/Semantik-/Hybrid‑Clustering für alle Dev‑Sessions mit `main.py` oder wahlweise nur eine Session über den Wrapper `run_time_semantic_hybrid_clustering`.
2. Gaze‑basiertes Post‑Processing mit `post_step.main` zur Erzeugung von Distanzmatrizen, Agglo‑ und spektralen Gaze‑Clustern sowie globalen JSON‑Summaries.
3. Optional: ASD‑Overlap‑Baseline, Seating‑Baselines und Video‑Cluster‑Quality‑Eval zum Vergleich der Video‑basierten Ansätze mit Audio/ASD‑basierten Verfahren.
4. Visualisierung der Resultate (z. B. Distanz‑Histogramme und Correct‑Rate‑Kurven) mit `plots_py.py` und den geladenen JSON‑Summaries.

Wo konkrete Datenpfade nötig sind, werden Platzhalter wie `"/path/to/chime/data"` verwendet; diese müssten im Challenge‑Setup an die lokale Ordnerstruktur angepasst werden.


In [ ]:
# Mini-Pipeline-Run-Beispiel

def run_full_video_pipeline_for_dev():
    """
    Demonstration of an end-to-end run for the dev set:

    1. Time/semantic/hybrid clustering via main.py.
    2. Gaze-based post-processing via post_step.py.
    3. Optional ASD overlap / seating / video quality evals.
    4. Load global JSONs for subsequent plotting. [file:2][file:3][file:4][file:5][file:6][file:8][file:9][file:10][file:14]
    """
    # 1) Conversation clustering (time / semantic / hybrid).
    run_time_semantic_hybrid_clustering(
        session_glob=str(DEV_SESSIONS_ROOT / "session*"),
        output_dirname="output_semantic",
        max_length=15,
    )

    # 2) Gaze-basiertes Post-Processing (Seat/Gaze-Distanzen, Agglo & Spectral). 
    run_post_step(
        output_root=OUTPUT_GAZE_ROOT,
        labels_root=DEV_SESSIONS_ROOT,
    )

    # 3) Optionale weitere Evals zum Vergleich der Videoansätze.
    # Achtung: eval_asd_overlap_baseline und eval_seating arbeiten intern
    # mit TEAMC_DATA_BIN/TEAMC-spezifischen Konstanten; 
    #
    # run_asd_overlap_baseline()
    # run_seating_baseline_eval()
    # run_video_cluster_quality_eval()

    # 4) Globale JSONs laden, z.B. für Plotting. 
    data = load_global_distance_and_eval()
    return data



## Hinweise zur Evaluation & Weiterarbeit

Die erzeugten Outputs dienen in der CHiME‑9‑Pipeline als Gesprächs‑Clusterings, die weiter in ASR‑ oder Diarisations‑Komponenten eingespeist werden können (z. B. zur Sprecher‑Weisung der Audio‑Kanäle oder für turn‑basierte Transkript‑Analyse).
Die JSON‑Formate `speakertocluster_*.json` folgen dabei konsistent dem Schema `{"spk0": 0, "spk1": 1, ...}`, was eine einfache Integration in nachgelagerte Tools ermöglicht.
Für Anpassungen an andere Sessions oder Konfigurationen bieten sich folgende Stellschrauben an:

- **Session-Auswahl:** Änderung des Glob‑Patterns in `run_time_semantic_hybrid_clustering` (z. B. `data-bin/dev_central_videos/session*`) oder explizite Liste von Session‑Ordnern. 
- **Datenwurzeln:** Anpassung von `PROJECT_ROOT` und `DATABIN_ROOT` an lokale Pfade bzw. andere Team‑Verzeichnisse (z. B. `team_a`, `team_b`). 
- **Gaze-Parameter:** Die Konstanten `YAWSIGN`, `GAZE_MIN_SAMPLES`, `BASE_GAZE_TOL_DEG`, `MAX_GAZE_TOL_DEG`, `WMUTUAL` und `WONEWAY` steuern, wie stark und wie tolerant Gaze in die Distanzmatrizen einfließt; Änderungen sollten jedoch direkt in `post_step.py` erfolgen und sorgfältig evaluiert werden.
- **Spektral-Parameter:** `SPECTRAL_KMIN`, `SPECTRAL_KMAX`, `SPECTRAL_KNN` und `SPECTRAL_GAMMA` definieren k‑Suche, kNN‑Sparsifizierung und Graph‑Schärfung für die spektralen Gaze‑Cluster.

Für weiterführende Experimente kann das Notebook als zentraler Einstiegspunkt dienen, um neue Varianten (z. B. modifizierte Similaritätsdefinitionen, alternative k‑Auswahlstrategien oder zusätzliche Video‑Features) prototypisch zu testen, während die bestehende, challenge‑konforme Logik der Skripte unangetastet bleibt.
